In [1]:
"""
Clip morphology rasters to extent of meteorological data
"""
#py_data_analysis
import os
from osgeo import gdal
import numpy as np
import glob
import xarray as xr
from osgeo import gdal
import matplotlib.pyplot as plt

#### Method 1: Using a Region extent

In [ ]:
# Load reference grid
mHM_grid = xr.open_dataset(r"W:\VUB\_main_research\mHM\mhm_belgium\belgium\lai\lai.nc", decode_times=False)

# Define directories
in_dir = r'W:\VUB\_main_research\mHM\mhm_belgium\belgium\morph'
out_dir = r'W:\VUB\_main_research\mHM\mhm_belgium\RMI\morph'
os.makedirs(out_dir, exist_ok=True)

# Get list of .asc files
morph_files = glob.glob(os.path.join(in_dir, '*.asc'))

# Define clipping bounds, rounded to the nearest 0.5
lat_min = 49.5
lat_max = 51.5
lon_min = 2.5
lon_max = 6.5

# Clip files
for morph_file in morph_files:
    out_file = os.path.join(out_dir, os.path.basename(morph_file))
    gdal.Warp(out_file, morph_file, format='AAIGrid', outputBounds=[lon_min, lat_min, lon_max, lat_max])
    print(f'Clipped: {out_file}', end='\r')

#### Method 2: Using a Raster Extent

This option clips rasters based on smaller raster that only covers the study domain e.g the boundaries of a country

In [32]:
import os
import rasterio

# Paths
input_path = r'W:\VUB\_main_research\mHM\mhm_belgium\RMI\morph'
output_path = r'W:\VUB\_main_research\mHM\mhm_belgium\RMI\morph_BE_domain'
domain_raster_path = r'W:\VUB\_main_research\mHM\mhm_belgium\RMI\morph_BE_domain\be_dem.asc'

# List all .asc files in the input directory
raster_list = [f for f in os.listdir(input_path) if f.endswith('.asc')]

# Read the domain raster once (the mask)
with rasterio.open(domain_raster_path) as src_domain:
    mask_data = src_domain.read(1)
    mask_meta = src_domain.meta

# For each raster, open, mask, and write output
for raster_name in raster_list:
    input_raster = os.path.join(input_path, raster_name)
    output_raster = os.path.join(output_path, raster_name)
    
    with rasterio.open(input_raster) as srcA:
        A = srcA.read(1)  # read band 1
        
        # Make a copy of A to hold the masked result
        masked_data = A.copy()
        
        # Wherever domain raster == -9999, set output to -9999
        # (i.e., mask out those pixels in the input raster)
        masked_data[mask_data == -9999] = -9999
        
        # Copy and update metadata
        out_meta = srcA.meta.copy()
        out_meta.update({
            "nodata": -9999,
            "dtype": "float32"  # or match your data type
        })
    
    # Save the masked raster
    with rasterio.open(output_raster, 'w', **out_meta) as dst:
        dst.write(masked_data, 1)

print("Masking complete!")


Masking complete!
